# .tflite experiments

## Building flatc

- Visit the [Flatbuffer repository](https://github.com/google/flatbuffers)
- Tutorials are [here](https://google.github.io/flatbuffers/flatbuffers_guide_building.html)
```
git clone https://github.com/google/flatbuffers
cmake -G "Unix Makefiles" -DCMAKE_BUILD_TYPE=Release
make
```

## Generate TF Lite schema
```
wget https://raw.githubusercontent.com/tensorflow/tensorflow/master/tensorflow/lite/schema/schema.fbs
pip install flatbuffers
./flatbuffers/flatc -p schema.fbs
```
The generated python code is under `tflite/`

## Generate the .tflite File

We are experimenting with a network with its architecture shown below: ![](https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/image/mlp_mnist.png)

The code to construct, train and export the model as a `.tflite` can be viewed [here](https://gist.github.com/neil-tan/6d77ee4029dff09d33830d716ac26a3a).
The pre-generated TF Lite file is included in the current repository as `converted_model.tflite`.




## Python Experiments
Referencing [tutorial1](https://google.github.io/flatbuffers/flatbuffers_guide_tutorial.html) and [tutorial2](https://google.github.io/flatbuffers/flatbuffers_guide_use_python.html)

Flatbuffer Import

In [1]:
import flatbuffers
from tflite.Model import Model

In [2]:
buf = open('converted_model.tflite', 'rb').read()
buf = bytearray(buf)
model = Model.GetRootAsModel(buf, 0)
print(model.Version())

3


In [3]:
# getting subgraph(s)
subgraphs_len = model.SubgraphsLength()
print(subgraphs_len)

1


In [4]:
subgraph = model.Subgraphs(0)
print(subgraph)

Operators

In [5]:
#op code lookup object
op_codes = []
for i in range(0, model.OperatorCodesLength()):
    op_code =  model.OperatorCodes(i)
    op_codes.append(op_code)
    
print(op_codes)

[<tflite.OperatorCode.OperatorCode object at 0x10cb56438>, <tflite.OperatorCode.OperatorCode object at 0x10cb56a38>]


In [6]:
# printing the built-in op codes
# see BuiltinOperator.py

from tflite.BuiltinOperator import BuiltinOperator
from tflite.BuiltinOptions import BuiltinOptions

builtin_ops = {v: k for k, v in BuiltinOperator.__dict__.items()}
op_options = {v: k for k, v in BuiltinOptions.__dict__.items()}


In [7]:
print(subgraph.OperatorsLength(), "operators:")

for i in range(0, subgraph.OperatorsLength()):
    op = subgraph.Operators(i)
    opIndex = op.OpcodeIndex()
    op_code = op_codes[opIndex]
    builtin_code = op_code.BuiltinCode()
    op_type = builtin_ops[builtin_code]
    print(op_type)
    
    if(op_type == 'FULLY_CONNECTED'):
        option = None
        custom_option = None
        
        if(op.CustomOptionsLength() < 1):
            #TODO: abstract these 2 lines
            from tflite.FullyConnectedOptions import FullyConnectedOptions
            option = FullyConnectedOptions()
            builtin_data = op.BuiltinOptions()
            option.Init(builtin_data.Bytes, builtin_data.Pos)
            from tflite.FullyConnectedOptionsWeightsFormat import FullyConnectedOptionsWeightsFormat
            w_formats = {v: k for k, v in FullyConnectedOptionsWeightsFormat.__dict__.items()}
        else:
            custom_option = op.CustomOptionsAsNumpy()
            #print("custom option format: ", op.CustomOptionsFormat())
        
        print(w_formats[option.WeightsFormat()])
        
        #input
        input_tensors = [subgraph.Tensors(input_idx) for input_idx in op.InputsAsNumpy()]
        for tensor in input_tensors:
            print(tensor.Name(), ", ", tensor.ShapeAsNumpy(), ", ", tensor.Type())
            
    print()
        

        

4 operators:
FULLY_CONNECTED
DEFAULT
b'x' ,  [  1 784] ,  0
b'Variable/transpose' ,  [128 784] ,  3
b'MatMul_bias' ,  [128] ,  0

FULLY_CONNECTED
DEFAULT
b'Relu' ,  [  1 128] ,  0
b'Variable_2/transpose' ,  [ 64 128] ,  3
b'MatMul_1_bias' ,  [64] ,  0

FULLY_CONNECTED
DEFAULT
b'Relu_1' ,  [ 1 64] ,  0
b'Variable_4/transpose' ,  [10 64] ,  0
b'MatMul_2_bias' ,  [10] ,  0

ARG_MAX

